# Bullet Forge Fine-tuning

This notebook is used for fine-tuning the Bullet Forge model in Open AI's API. Please refer to the README for more details.

## Step 1: Provide Required Arguments

Make sure to provide the following arguments:
- JSONL_FILE: The path to the JSONL file containing the training data.
- BASE_MODEL: The base model to use for fine-tuning.

In [3]:
JSONL_FILE = input("Enter the path to the JSON file: ")
BASE_MODEL = input("Enter the base model: ")

## Step 2: Prepare Training Data

Prepare the training data using the provided JSON file.

In [ ]:
print("==> Preparing training data...")
PREPARED_DATA = input("Enter the prepared data: ")
if not PREPARED_DATA:
    raise SystemExit(1)
print("==> Training data preparation completed!")

## Step 3: Provide Open AI API Key

Insert your own Open AI API key.

In [ ]:
API_KEY = input("Enter the Open AI API key: ")
%env OPENAI_API_KEY $API_KEY

## Step 4: Start the Fine-Tuning Job

Once the request is made to the API, the fine-tuning may take a while to complete. The following will check for completion periodically.

In [ ]:
import time

print("==> Starting fine-tuning job...")
FINE_TUNED_MODEL = input("Enter the fine-tuned model: ")
if not FINE_TUNED_MODEL:
    raise SystemExit(1)
print("==> Fine-tuning job started.")

print("==> Waiting for fine-tuning to complete...")
while True:
    JOB_STATUS = input("Enter the job status: ")
    if JOB_STATUS == "succeeded":
        break
    # Sleep for 10 seconds before checking again
    time.sleep(10)
print("==> Fine-tuning completed!")


## Step 5: Use The Fine-Tuned Model

Fine-tuned model usage: `openai api completions.create -m <FINE_TUNED_MODEL> -p <YOUR PROMPT>`
